In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark=SparkSession.builder.appName('exer_spark').getOrCreate()

## Running the Average Friends by Age ##

In [32]:
f_data=sc.textFile('file:///D:\\spark-2.3.2-bin-hadoop2.7\\fakefriends.csv')
f_data.take(5)


['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [8]:
age_numf=f_data.map(lambda x:(x.split(',')[2],x.split(',')[3]))

In [31]:
age_num_map=age_numf.mapValues(lambda x: (x,1))
age_num_map.take(5)

[('33', ('385', 1)),
 ('26', ('2', 1)),
 ('55', ('221', 1)),
 ('40', ('465', 1)),
 ('68', ('21', 1))]

In [30]:
age_totalnum=age_num_map.reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1]))
age_totalnum.take(5)

[('33', ('38574471275245356460294243463228410', 12)),
 ('26', ('228184282381145345293298492269254738312439184', 17)),
 ('40', ('4652544594071828438934940619817233567261286220', 17)),
 ('68', ('21264112490481217189206293423', 10)),
 ('54', ('30725375440744123536939746272442115', 13))]

In [24]:
average_age=age_totalnum.mapValues(lambda x:float(x[0])/x[1])

average_age.take(5)

[('33', 3.214539272937113e+33),
 ('26', 1.3422604845949725e+43),
 ('40', 2.736790937689311e+44),
 ('68', 2.1264112490481217e+27),
 ('54', 2.3634904185187785e+33)]

## wordcount of a text file with regex ##

In [296]:
lines=sc.textFile('file:///D:\\spark-2.3.2-bin-hadoop2.7\\Book.txt')
#words=lines.flatMap(lambda x:x.split())
#words.take(10) # there are many special characters and blanks (seen thru collect()),so we need to use regex to ensure only valid words are selected

        

['Self-Employment:',
 'Building',
 'an',
 'Internet',
 'Business',
 'of',
 'One',
 'Achieving',
 'Financial',
 'and']

In [309]:
import re

def regex_pr(line): # to filter special characters
    c=re.compile(r'\W+',re.UNICODE)
    return c.split(line.lower())  # after compiling regexp for detecting valid chars apply that to 
                                  # split line where capitals and smalls are treated equal. 
    #return re.split(c,line.lower())
    #re.split(<pattern,<text>) expects a text s argument.
    #re.compile(<pattern>,<flag>) and saving the resulting regular expression object for reuse is more efficient
    #when the expression will be used several times in a single program
    # re.UNICODE to specify the text has some unicode info . you can also write it as re.U.
 
   

In [310]:
words=lines.flatMap(regex_pr)  
words.take(10)

['self',
 'employment',
 'building',
 'an',
 'internet',
 'business',
 'of',
 'one',
 'achieving',
 'financial']

In [292]:
count_of_words=words.map(lambda x:(x,1)).reduceByKey(lambda x,y:x+y)
count_of_words.take(5)


[('self', 111), ('an', 178), ('internet', 26), ('business', 383), ('of', 970)]

In [293]:
for c in count_of_words.take(5): # printing count of words
    print(c) 

('self', 111)
('an', 178)
('internet', 26)
('business', 383)
('of', 970)


In [102]:
sort_by_count=count_of_words.map(lambda x :(x[1],x[0])).sortByKey(False)
sort_by_count.first() # max count and related word

(1878, 'you')

In [294]:
for c in count_of_words.take(5): # formating result as word:count
    count = str(c[1])
    word = str(c[0])
    print(word+':'+count)


self:111
an:178
internet:26
business:383
of:970


## Find the Most Popular Superhero and coappearances in a Social Graph ##

In [157]:
def parseNames(line):
    fields = line.split('\"') #split on quotes to get key and name
    return (int(fields[0]), fields[1])

In [171]:
def countCoOccurences(line):
    #elements = line.strip().split()
    elements = line.split()
    return (int(elements[0]), len(elements) -1) # to subtract id (key) from list

In [163]:
names = sc.textFile("file:///D:\\spark-2.3.2-bin-hadoop2.7\\marvel-names.txt")
names.take(5)

['1 "24-HOUR MAN/EMMANUEL"',
 '2 "3-D MAN/CHARLES CHAN"',
 '3 "4-D MAN/MERCURIO"',
 '4 "8-BALL/"',
 '5 "A"']

In [188]:
namesRdd = names.map(parseNames)
namesRdd.take(10) # contains key of superhero,name

[(1, '24-HOUR MAN/EMMANUEL'),
 (2, '3-D MAN/CHARLES CHAN'),
 (3, '4-D MAN/MERCURIO'),
 (4, '8-BALL/'),
 (5, 'A'),
 (6, "A'YIN"),
 (7, 'ABBOTT, JACK'),
 (8, 'ABCISSA'),
 (9, 'ABEL'),
 (10, 'ABOMINATION/EMIL BLO')]

In [164]:
lines = sc.textFile("file:///D:\\spark-2.3.2-bin-hadoop2.7\\marvel-graph.txt")
lines.take(5)

['5988 748 1722 3752 4655 5743 1872 3413 5527 6368 6085 4319 4728 1636 2397 3364 4001 1614 1819 1585 732 2660 3952 2507 3891 2070 2239 2602 612 1352 5447 4548 1596 5488 1605 5517 11 479 2554 2043 17 865 4292 6312 473 534 1479 6375 4456 ',
 '5989 4080 4264 4446 3779 2430 2297 6169 3530 3272 4282 6432 2548 4140 185 105 3878 2429 1334 4595 2767 3956 3877 4776 4946 3407 128 269 5775 5121 481 5516 4758 4053 1044 1602 3889 1535 6038 533 3986 ',
 '5982 217 595 1194 3308 2940 1815 794 1503 5197 859 5096 6039 2664 651 2244 528 284 1449 1097 1172 1092 108 3405 5204 387 4607 4545 3705 4930 1805 4712 4404 247 4754 4427 1845 536 5795 5978 533 3984 6056 ',
 '5983 1165 3836 4361 1282 716 4289 4646 6300 5084 2397 4454 1913 5861 5485 ',
 '5980 2731 3712 1587 6084 2472 2546 6313 875 859 323 2664 1469 522 2506 2919 2423 3624 5736 5046 1787 5776 3245 3840 2399 ']

In [172]:
pairings = lines.map(countCoOccurences)
pairings.take(10)

[(5988, 48),
 (5989, 40),
 (5982, 42),
 (5983, 14),
 (5980, 24),
 (5981, 17),
 (5986, 142),
 (5987, 81),
 (5984, 41),
 (5985, 19)]

In [177]:
totalFriendsByCharacter = pairings.reduceByKey(lambda x, y : x + y)


[(48, 5988),
 (42, 5982),
 (24, 5980),
 (142, 5986),
 (41, 5984),
 (13, 6294),
 (42, 270),
 (45, 272),
 (410, 274),
 (15, 276)]

In [313]:
mostpopular=totalFriendsByCharacter.sortBy(lambda x:x[1],False).first() #sort value for each key in descending order
mostpopular #key of superhero,occurences

(859, 1933)

In [214]:
# alternate way of finding max value of key is using max()

flipped = totalFriendsByCharacter.map(lambda x : (x[1],x[0]))
flipped.take(10)
mostpopular1=flipped.max() # max number of coappearances ,max() operate on key of k,v pair or max(<key function>) where key can be changed
mostpopular1 #contains count of friends,key of superhero

(1933, 859)

In [218]:
# to get matching name from id (key

#for name in namesRdd.collect():
    #if mostpopular1[1]==name[0]: # mostpopular[0]==name[0]:
       #print(name[1])
        
#alternate way 

print('popular name is '+namesRdd.lookup(mostpopular[0])[0]+' with coappearances='+ str(mostpopular[1])) # lookup(<value>),to lookup for a value in rdd. Return list of matches if value found
                                                                                                         #[0] after lookup() - to display value of list

#print('popular name is '+namesRdd.lookup(mostpopular1[1])[0]+' with coappearances='+ str(mostpopular1[0]))




popular name is CAPTAIN AMERICA with coappearances=1933


## Total-Amount-By-Customer##

In [219]:
cust = sc.textFile("file:///D:\\spark-2.3.2-bin-hadoop2.7\\customer-orders.csv")

In [226]:
cust.take(5)


['44,8602,37.19',
 '35,5368,65.89',
 '2,3391,40.64',
 '47,6694,14.98',
 '29,680,13.08']

In [241]:
def parseLine(line):
    fields = line.split(',')
    customerId = int(fields[0])
    itemId = fields[1]
    itemAmt = float(fields[2])
    return (customerId, itemAmt)

In [257]:
#orders = cust.map(parseLine)
id_amt=cust.map(lambda line: (int(line.split(',')[0]),float(line.split(',')[2])))
totals = id_amt.reduceByKey(lambda x, y: x + y)
totals.take(5)

[(44, 4756.890000000001),
 (2, 5994.59),
 (70, 5368.249999999999),
 (14, 4735.030000000001),
 (42, 5696.840000000002)]

In [278]:
for result in totals.take(5):
    print("{:.2f}".format(result[1]),"\t{:.2f}".format(result[0])) #:.2f for formatting in placeholder and then using format() to display result

4756.89 	44.00
5994.59 	2.00
5368.25 	70.00
4735.03 	14.00
5696.84 	42.00


## Min/Max -Temperatures per station ID##

In [316]:
lines = sc.textFile("file:///D:\\spark-2.3.2-bin-hadoop2.7\\1800.csv")
lines.take(5)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,']

In [333]:
# To take off required fields and assigning them to variable,create function where split line on a pattern and assign variables to reqd fields.

def parseLine(line):
    fields = line.split(',')
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationID, entryType, temperature)


In [335]:
parsedLines=lines.map(parseLine) # use the function to actual line element of rdd
parsedLines.take(5)

[('ITE00100554', 'TMAX', 18.5),
 ('ITE00100554', 'TMIN', 5.359999999999999),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 16.52),
 ('EZE00100082', 'TMIN', 7.699999999999999)]

In [359]:
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1]) # check if TMIN as a 1st pos value in each element of rdd,hence use filter() to filter out such elements of rdd
minTemps.take(5)

[('ITE00100554', 'TMIN', 5.359999999999999),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMIN', 9.5),
 ('EZE00100082', 'TMIN', 8.599999999999998),
 ('ITE00100554', 'TMIN', 23.72)]

In [360]:
id_temp=minTemps.map(lambda x:(x[0],x[2]))
id_temp.take(5)

[('ITE00100554', 5.359999999999999),
 ('EZE00100082', 7.699999999999999),
 ('ITE00100554', 9.5),
 ('EZE00100082', 8.599999999999998),
 ('ITE00100554', 23.72)]

In [361]:
minTemps = id_temp.reduceByKey(lambda x,y: x if x<y else y) # to calculate min of values (temps) per key
minTemps.collect()

[('ITE00100554', 5.359999999999999), ('EZE00100082', 7.699999999999999)]

In [376]:
for m in minTemps.collect(): #display min temperature per station
    print('station Id '+m[0]+' has min temp of {:.2f}F'.format(m[1]))
    
#to get lowest temperature and related station ID and of all stations.

m=minTemps.min(lambda x:x[1]) # to get min temperature pair

print('\nstation Id '+m[0]+' has lowest temp of {:.2f}F '.format(m[1])+'of all stations')

station Id ITE00100554 has min temp of 5.36F
station Id EZE00100082 has min temp of 7.70F

station Id ITE00100554 has lowest temp of 5.36F of all stations
